In [ ]:
# default_exp dataloader

# dataloader

> API details.

1. read_image(path: str, channels: int=3)
2. clf.load_from_folder
3. clf.load_from_csv
4. detect.load_from_xml
5. detect.load_from_csv
6. detect.load_from_json
7. detect.load_from_tfrecord

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import tensorflow as tf
import pathlib
import os

from typing import Union

from chitra.core import remove_dsstore
from chitra.image import read_image, resize_image


In [ ]:
#export
AUTOTUNE = tf.data.experimental.AUTOTUNE

# CLF

``DataLoader class for loading dataset for image classification tasks.``
All private functions use primitive datatypes


## clf.load_from_folder
## clf.load_from_csv

In [ ]:
#export
def get_basename(path: tf.string):
    assert isinstance(path, tf.Tensor)
    return tf.strings.split(path, os.path.sep)[-1]

In [ ]:
#export
class Clf(object):
    def __init__(self):
        self.CLASS_NAMES = None
    
    def _get_image_list(self, path: str):
        """`path`: pathlib.Path
        Returns: list of images
        """
        assert isinstance(path, str)
        list_images = tf.data.Dataset.list_files(f'{path}/*/*')
        return list_images
    
    def _process_path(self, path:str, size:Union[None, tuple] = None):
        """`path` :str
        `size`: None or tuple
        """
        assert isinstance(path, (str, tf.Tensor)), f'type of path is {type(path)}, expected type str'
        img = read_image(path)
        img = tf.py_function(resize_image, [img, (160, 160)], [tf.float32])
        
        label = tf.strings.split(path, os.path.sep)[-2]
        return img, label

    
    def from_folder(self, path: Union[str, pathlib.Path]):
        """Load dataset from given path.
        Args:
            path: string, path of folder containing dataset.
        Returns: tf.data.Dataset
        """
        assert isinstance(path, (str, pathlib.Path))
        path = pathlib.Path(path)
        remove_dsstore(path)
        
        list_folders = tf.data.Dataset.list_files(str(path/'*'))
        list_images = self._get_image_list(str(path))
                
        self.CLASS_NAMES = tuple(get_basename(e).numpy() for e in list_folders)
        
        data = list_images.map(self._process_path, num_parallel_calls=AUTOTUNE)
        return data

In [ ]:
#hide
clf = Clf()
data = clf.from_folder(cats_path)
data

<DatasetV1Adapter shapes: (<unknown>, ()), types: (tf.float32, tf.string)>

In [ ]:
#hide
for e in data.take(1): print(e)

(<tf.Tensor: id=3337, shape=(1, 160, 160, 3), dtype=float32, numpy=
array([[[[ 43.340626,  39.340626,  27.340626],
         [ 36.89375 ,  32.89375 ,  21.89375 ],
         [ 35.553123,  31.553125,  22.553125],
         ...,
         [160.7875  , 134.7875  ,  85.7875  ],
         [159.23438 , 133.23438 ,  82.234375],
         [143.55313 , 120.55312 ,  50.553123]],

        [[ 37.340626,  33.340626,  21.340626],
         [ 38.021877,  34.021877,  23.021875],
         [ 43.659374,  39.659374,  30.659374],
         ...,
         [151.31876 , 123.      ,  67.34062 ],
         [151.31876 , 123.34062 ,  67.      ],
         [143.34062 , 117.3625  ,  48.021877]],

        [[ 39.765625,  35.765625,  23.765625],
         [ 41.765625,  37.765625,  26.765625],
         [ 40.0625  ,  36.0625  ,  27.0625  ],
         ...,
         [148.46875 , 118.703125,  58.703125],
         [147.9375  , 118.171875,  58.171875],
         [141.23438 , 111.703125,  49.      ]],

        ...,

        [[ 61.234375,  5

In [ ]:
#hide
data.element_spec

(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None),
 TensorSpec(shape=(), dtype=tf.string, name=None))